In [1]:
from flask import Flask, request, jsonify
import requests
import json
import os
from difflib import get_close_matches
from openai import OpenAI

# This assumes you have a config.py file with your tokens.
from config import PAGE_ACCESS_TOKEN, VERIFY_TOKEN, OPENAI_API_KEY

In [2]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
app = Flask(__name__)

In [4]:
# --- Load data from files ---
# It's a good practice to handle potential FileNotFoundError
try:
    with open("faq.json", "r", encoding="utf-8") as f:
        faq_raw = json.load(f)
    faq_database = {k.lower().strip(): v for k, v in faq_raw.items()}

    with open("general_info.txt", "r", encoding="utf-8") as f:
        general_info = f.read()
except FileNotFoundError as e:
    print(f"Error: {e}. Please make sure 'faq.json' and 'general_info.txt' exist in the same directory.")
    faq_database = {}
    general_info = ""

In [5]:
# --- Root route for testing ---
@app.route("/", methods=["GET"])
def home():
    return "Messenger bot is running!"

In [6]:
# --- Messenger helper ---
def send_message(recipient_id, text):
    """Send message back to a Facebook Messenger user."""
    url = "https://graph.facebook.com/v21.0/me/messages"
    params = {"access_token": PAGE_ACCESS_TOKEN}
    headers = {"Content-Type": "application/json"}
    data = {"recipient": {"id": recipient_id}, "message": {"text": text}}
    r = requests.post(url, params=params, headers=headers, json=data)
    if r.status_code != 200:
        print(f"Error sending message: {r.text}")
        print(f"Facebook API Response: {r.text}")

In [7]:
# --- GPT fallback ---
def generate_human_like_response(user_question):
    """
    Generates a response using OpenAI as a fallback for unknown questions.
    """
    faq_text = "\n".join([f"Q: {q}\nA: {a}" for q, a in faq_database.items()])
    prompt = f"""You are a friendly chatbot. Here is general information about you:{general_info}Here is a FAQ that you should use as a reference:{faq_text}Answer the user question in a natural, human-like way.User question: {user_question}Answer:"""
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()

In [8]:
# --- Webhook ---
@app.route("/webhook", methods=["GET", "POST"])
def webhook():
    if request.method == "GET":
        # Webhook verification
        token_sent = request.args.get("hub.verify_token")
        return request.args.get("hub.challenge") if token_sent == VERIFY_TOKEN else "Invalid token"
    else:
        # Handle messages
        output = request.get_json()
        for event in output.get("entry", []):
            for messaging_event in event.get("messaging", []):
                sender_id = messaging_event["sender"]["id"]
                if messaging_event.get("message"):
                    user_text = messaging_event["message"].get("text")
                    if user_text:
                        user_text_lower = user_text.lower().strip()

                        # 1️⃣ Exact match
                        response_text = faq_database.get(user_text_lower)

                        # 2️⃣ Fuzzy match
                        if not response_text:
                            matches = get_close_matches(user_text_lower, faq_database.keys(), n=1, cutoff=0.6)
                            if matches:
                                response_text = faq_database[matches[0]]

                        # 3️⃣ GPT fallback
                        if not response_text:
                            response_text = generate_human_like_response(user_text)

                        send_message(sender_id, response_text)
        return "ok", 200

In [9]:
# --- Run server ---
if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Sep/2025 00:11:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:12:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:12:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Sep/2025 00:12:20] "GET /webhook?hub.mode=subscribe&hub.challenge=1434312921&hub.verify_token=kdjjskhfjskfnklafmkasa6s45asffsdnfsdf5dsfc HTTP/1.1" 200 -
